<div style="border-style: solid;border-width: 1px;">
<div style="height: 200px; 
            width: 750px; 
            color: #ffffff;
            margin: auto;
	        background-color: #ff6600; 
	        background-image: url(http://icarus.lngs.infn.it/img/n3.jpg); 
            background-repeat: no-repeat;
            background-position: center;">
</div>

<div style="text-align:center">
    <h1><font color="blue"><font size="6">Study of 2D Noise Filtering Techniques in ICARUS</font></font></h1><br>
    <font color="gray"><font size="3">A jupyter notebook aimed at studying 2D Noise Filtering with the ICARUS LAr TPC</font></font><br>
</div>
</div>
<br>
<h3>What this notebook intends to do:</h3>
<ul>
    <li>Operates on art root format data files that contain raw::RawDigit objects</li>
        <ul>
            <li>The RawDigits contain waveforms representing the TPC readout </li>
        </ul>
    <li>Once the RawDigits have been recovered the idea is to analyze the raw data</li>
        <ul>
            <li>Look at various algorithms for reducing noise while maintaining signal using 2D techniques</li>
        </ul>
</ul>

<h3>What is needed to run this notebook:</h3>
<ul>
    <li><p>Necessary python libraries/includes</p>
        <ul>
            <li> matplotlib   - the basic plotting package though note that here we mostly use plot-ly</li>
            <li> numpy        - python array handling and loads of useful functions</li>
            <li> scipy        - where we get the analytic functions we will make heavy use of </li>
            <li> ipywidgets   - useful for making interactive output</li>
            <li> uproot       - see below</li>
            <li> plotly       - the plot-ly graphing package which allows interactive plots</li>
        </ul></li>
    <li>uproot<br>
        In this notebook we use "uproot" to handle the root input. For more information on uproot follow this <a href="https://uproot.readthedocs.io/en/latest/">link</a></li>
</ul>



<div style="text-align:center;border-style: solid;border-width: 1px;">
    <h2><font color="blue"><font size="5">Start Up Block</font></font></h2><br>
</div>

<ul>
    <li>Import of standard libraries used below</li>
    <li>Set the system path to include looking in the package of noise handling functions/objects</li>
    <li>We define two variables which define the location of the input file</li>
        <ul>
            <li> PATHNAME       - Fully qualified path to the folder containing the input file</li>
            <li> RECOFILENAME   - the fully qualified file name for the input data file</li>
        </ul>
    <li>With these definitions we can open and read the input data file</li>
</ul>

In [ ]:
#
# Standard set up block to start with
#
# Probably most people will simply want "matplotlib inline" but on a mac it is useful to do the other stuff
%matplotlib inline
##matplotlib qt
##matplotlib notebook
##config InlineBackend.figure_format ='retina'
#%pylab

# This allows reading of root files without having to actually use root. Nice!
import uproot

# Useful stuff
import numpy as np
import math
import plotly.graph_objects as go
import plotly.subplots as subplots

from plotly.colors import DEFAULT_PLOTLY_COLORS

###############################################################################
# TODO:
# Set the path to our library of functions/objects/etc
import sys

sigProcPath = "/LArSoft/ICARUS/workarea/artroot/icarus-sigproc-tools"
sys.path.insert(0,sigProcPath)

###############################################################################
# TODO:
# Set the path and name of the data file to be read
PATHNAME       = "/LArSoft/ICARUS/workarea/artroot"
RECOFILENAME   = PATHNAME + "/data_run621_1_EW05_PED_20190926T201608_DECODE.root"
#RECOFILENAME   = PATHNAME + "/data_run618_2_EW01M_PED_20190926T174904_DECODE.root"

<div style="text-align:center;border-style: solid;border-width: 1px;">
    <h2><font color="blue"><font size="5">Open, Read and Process the Input File</font></font></h2><br>
</div>

<ul>
    <li>We define three variables which should be common to all files
        <ul>
            <li> RECOFOLDERNAME - the name of the folder for the particular data set we want to analyze</li>
            <li> DAQNAME.       - currently unused</li>
        </ul></li>
    <li>With these definitions we can open and read the input data file</li>
    <li>Contents of the input file are read into a numpy array</li>
    <li>Once read in, then go ahead and process the input file with the standard noise filter</li>
</ul>

In [ ]:
from sigproc_tools.sigproc_objects.filterevents import FilterEvents

# Below should be standard for the test data files currently available
RECOFOLDERNAME = "Events" 
DAQNAME        = "raw::RawDigits_daq__TPCANALYSIS."

# Grab the pandas dataframe from the input file for the tree we want to look at
print("Opening file: ",RECOFILENAME)
data_file = uproot.open(RECOFILENAME)

#print(data_file.keys(),"\n------\n")

print("Opening the folder contianing the RawDigits information: ",RECOFOLDERNAME)
events_folder = data_file[RECOFOLDERNAME]

# Go ahead and filter the events
filter = FilterEvents(events_folder,DAQNAME)

# What are the basic operating parameters?
nChannelsPerGroup = 64   # <==== We choose this
numChannels       = filter.rawdigits.numChannels(0)
numGroups         = numChannels // nChannelsPerGroup

numEvents = filter.filterEvents(nChannelsPerGroup)

print("Noise processing complete with",numEvents)

<div style="text-align:center;border-style: solid;border-width: 1px;">
    <h2><font color="blue"><font size="5">Compare Power Spectra of Input Data to Pure White Noise</font></font></h2><br>
</div>

In [ ]:
from plotting.graphs import plotEventView2D
from sigproc_tools.sigproc_functions.fakeParticle import *
from sigproc_tools.sigproc_functions.noiseProcessing import *
from sigproc_tools.sigproc_functions.noiseAnalysis import *
from sigproc_tools.sigproc_functions.responseFunctions import *

# Define some constants
whiteRMSRaw  = 2.4

numTicks     = filter.rawdigits.numTicks(0)
tickWidth    = 0.4            # 0.4 microseconds/tick
maxFrequency = 1. / tickWidth # max frequency in MHz

waveformShape = (numChannels,numTicks)

# Create an event full of channels with white noise
whiteNoiseWaveform = np.zeros(waveformShape)

whiteNoiseWaveform = genWhiteNoiseWaveform(tickWidth,whiteRMSRaw,waveformShape)

elecResponseVec = electronicsReponse(np.arange(numChannels),tickWidth)
whiteNoiseVec   = np.random.normal(loc=0.,scale=whiteRMSRaw,size=numTicks)

# Little cross check here...
whiteLessPed,whitePed,whiteRMS = getPedestalsAndRMS(whiteNoiseWaveform)
checkLessPed,checkPed,checkRMS = getPedestalsAndRMS(whiteNoiseVec)

print("Cross check, white waveform RMS:",np.mean(whiteRMS,axis=-1),", raw:",checkRMS)

isItWhite = np.mean(whiteNoiseWaveform,axis=0)

figWhite = subplots.make_subplots(rows=3,cols=1,subplot_titles=("Electronics Response","White Noise","White Noise Convolved"),vertical_spacing=0.12)

figWhite.add_trace(go.Scatter(x=np.arange(numTicks),y=elecResponseVec,name="Electronics Response",line=dict(color=DEFAULT_PLOTLY_COLORS[0],width=2)),row=1,col=1)

figWhite.add_trace(go.Scatter(x=np.arange(numTicks),y=whiteNoiseVec,name="Input White Noise",line=dict(color=DEFAULT_PLOTLY_COLORS[0],width=2)),row=2,col=1)

figWhite.add_trace(go.Scatter(x=np.arange(numTicks),y=whiteLessPed[10,:],name="White Noise",line=dict(color=DEFAULT_PLOTLY_COLORS[0],width=1)),row=3,col=1)
figWhite.add_trace(go.Scatter(x=np.arange(numTicks),y=isItWhite,name="Average Noise",line=dict(color=DEFAULT_PLOTLY_COLORS[1],width=2)),row=3,col=1)

figWhite.update_layout(height=900,width=1000,title_text="Electronics Response and White Noise",title_font_size=30)
figWhite.show()

freqVec, totWavePowerVec = getPowerVec(filter.waveLessPedAll,maxFrequency)
freqVec, corWavePowerVec = getPowerVec(filter.waveLessCoherentAll,maxFrequency)
freqVec, wNoisePowerVec  = getPowerVec(whiteNoiseWaveform,maxFrequency)

# combine the power vecs
meanTotPower = np.mean(np.mean(totWavePowerVec,axis=1),axis=0)
meanCorPower = np.mean(np.mean(corWavePowerVec,axis=1),axis=0)

meanWhiteNoise = np.mean(wNoisePowerVec,axis=0)

print("MeanTotPower[0]:",meanTotPower[0],", meanCorPower[0]:",meanCorPower[0],", meanWhite[0]:",meanWhiteNoise[0])

figAllFFT = go.Figure()

figAllFFT.add_trace(go.Scatter(x=freqVec,y=meanTotPower,name="Total Power",line=dict(color=DEFAULT_PLOTLY_COLORS[0],width=1)))
figAllFFT.add_trace(go.Scatter(x=freqVec,y=meanCorPower,name="Corrected Power",line=dict(color=DEFAULT_PLOTLY_COLORS[1],width=1)))
figAllFFT.add_trace(go.Scatter(x=freqVec,y=meanWhiteNoise,name="White noise Power",line=dict(color=DEFAULT_PLOTLY_COLORS[2],width=1)))

figAllFFT.show()




<div style="text-align:center;border-style: solid;border-width: 1px;">
    <h2><font color="blue"><font size="5">Create a Fake Particle and Overlay on Test Data</font></font></h2><br>
</div>

<ul>
    <li>Generally, we can overlay a unipolar signal (simply a guassian) or bipolar (derivative of the gaussian)</li>
        <ul>
            <li>Upgrade: augment the signal processing to include the field response, then we can generate a delta function charge deposit for a given tick on a wire and convolve with field+electronics response</li>
        </ul>
    <li>Create a 2D channel x tick grid with the new "particle"</li>
    <li>Overlay (add) this to an existing waveform with noise</li>
    <li>Optionally, if you set "makePlots" to True then you can view a 2D event display</li>
</ul>

In [ ]:
# define parameters for out fake track
particleAngle = math.pi / 3.   # This is the angle between a wire and the track
                               # zero would be along the wire, pi/2 is perpendicular
startTick   = 100.
pulseHeight = 20.
pulseWid    = 8.
gaussParams = np.array([pulseHeight,0.,pulseWid])
pulseArea   = math.sqrt(2.*math.pi)*pulseHeight*pulseWid

maxProjection = 4.*pulseWid/math.sin(particleAngle)
lowStartTick  = startTick - maxProjection
hiStartTick   = startTick + maxProjection

#checkRange  = np.arange(-4.*pulseWid,4.*pulseWid,0.25)
#gaussPoints = gaussParticle(checkRange,*np.array([pulseHeight,0.,pulseWid]))

print("pulse height:",pulseHeight,", pulseWid:",pulseWid,", pulseArea:",pulseArea)
#print("integral:",0.25*np.sum(gaussPoints))
    
print("particleAngle:",particleAngle,',',math.tan(particleAngle),", maxProjection:",maxProjection)

# Remember that a tick is 0.4 us, drift velocity is 1.6 mm/us so one tick is ~0.64 mm
# Wire space is 3mm which means the distance between wires is ~4.7 ticks
channelRange  = np.arange(numChannels)
tickRange     = np.arange(numTicks)
channelCoords = 4.7 * channelRange / math.tan(particleAngle)
lowTicks      = channelCoords + lowStartTick
hiTicks       = channelCoords + hiStartTick

# Covention is that the last two dimensions of the waveform will be #channels x #ticks
particleArray = np.zeros(filter.waveLessCoherentAll.shape[-2:])

print("Confirming the shape of particleArray:",particleArray.shape)

# Now overlay the particle
for wireIdx in range(particleArray.shape[0]):
    gaussRange = np.arange(-maxProjection,maxProjection,1.)
    gaussVals  = gaussParticle(gaussRange,*gaussParams)
    tickRange  = np.rint(np.arange(lowTicks[wireIdx],hiTicks[wireIdx],1.)).astype(np.int)
    for tickIdx in range(len(tickRange)):
        particleArray[wireIdx][tickRange[tickIdx]] = gaussVals[tickIdx]
        
# Now overlay this on top of the input waveforms
waveformWithParticle = filter.waveLessPedAll[0,:,:] + particleArray

# And now make the event display
makePlots = False

if makePlots:
    overlayPic = plotEventView2D(waveformWithParticle)
else:
    print("Particle overlay onto waveform done")

In [ ]:
import scipy.ndimage as ndimage

median   = ndimage.median_filter(waveformWithParticle,size=4)
erosion  = ndimage.grey_erosion(median,size=(5,5))
dilation = ndimage.grey_dilation(median,size=(5,5))
opening  = ndimage.grey_dilation(erosion,size=(5,5))
gradient = dilation - erosion
average  = (dilation + erosion) / 2.

print("plotting")

gradPic = plotEventView2D(gradient)

